In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class PIDController:
    def __init__(self, Kp, Ki, Kd):
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        self.prev_error = 0
        self.integral = 0
        
    def compute(self, setpoint, pv, dt):
        error = setpoint - pv
        
        # Proportional term
        P = self.Kp * error
        
        # Integral term
        self.integral += error * dt
        I = self.Ki * self.integral
        
        # Derivative term
        derivative = (error - self.prev_error) / dt
        D = self.Kd * derivative
        
        # Save error for next derivative calculation
        self.prev_error = error
        
        # Compute output
        output = P + I + D
        
        return output

In [3]:
class GainScheduledPID:
    def __init__(self, pid_params):
        """pid_params is a list of (condition_function, Kp, Ki, Kd) tuples"""
        self.pid_params = pid_params
        self.current_pid = PIDController(0, 0, 0)
        self.prev_error = 0
        self.integral = 0
        
    def compute(self, setpoint, pv, dt):
        # Find appropriate parameters
        for condition, Kp, Ki, Kd in self.pid_params:
            if condition(setpoint, pv):
                self.current_pid.Kp = Kp
                self.current_pid.Ki = Ki
                self.current_pid.Kd = Kd
                break
                
        return self.current_pid.compute(setpoint, pv, dt)

# Example usage:
def low_temp_condition(setpoint, pv):
    return pv < 50

def high_temp_condition(setpoint, pv):
    return pv >= 50

pid_params = [
    (low_temp_condition, 30, 0.5, 5),  # more aggressive at low temps
    (high_temp_condition, 20, 0.2, 2)  # more conservative at high temps
]

scheduled_pid = GainScheduledPID(pid_params)